In [292]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, classification_report

In [293]:
def fetch_stock_data(ticker, start_date="1990-01-02 00:00:00-05:00", end_date=None):
    stock = yf.Ticker(ticker)
    stock_data = stock.history(start=start_date, end=end_date)
    stock_data.drop(["Dividends", "Stock Splits"], axis=1, inplace=True)
    return stock_data

In [294]:
def create_target_column(data, horizon=1):
    data["Tomorrow"] = data["Close"].shift(-horizon)
    data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
    return data

In [295]:
def train_random_forest_model(train_data, predictors, target):
    model = RandomForestClassifier(n_estimators=500, min_samples_split=50, random_state=1)
    model.fit(train_data[predictors], train_data[target])
    return model

In [296]:
def generate_additional_ticker_features(sp500, ticker_data, horizon, ticker):
    ticker_data = create_target_column(ticker_data, horizon=1)
    ticker_predictors = ["Close", "Volume", "Open", "High", "Low"]
        
    # Train model for the additional ticker
    ticker_model = train_random_forest_model(ticker_data, ticker_predictors, "Target")
    
    # Generate predictions for the additional ticker
    ticker_predictions = predict(ticker_data, ticker_data, ticker_predictors, ticker_model)
    
    # Create a new dataframe to store predictions for the additional ticker
    ticker_predictions_df = pd.DataFrame(index=sp500.index)
    ticker_predictions_df[f"Prediction_{ticker}"] = ticker_predictions["Predictions"]

    return ticker_predictions_df

In [297]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:, 1]
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [298]:
def generate_features(data, horizons):
    new_predictors = []

    for horizon in horizons:
        rolling_averages = data["Close"].rolling(horizon).mean()

        ratio_column = f"Close_Ratio_{horizon}"
        data[ratio_column] = data["Close"] / rolling_averages

        trend_column = f"Trend_{horizon}"
        data[trend_column] = data.shift(1).rolling(horizon).sum()["Target"]

        new_predictors += [ratio_column, trend_column]

    data = data.dropna(subset=data.columns[data.columns != "Target"])
    return data, new_predictors

In [299]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

In [300]:
def calculate_precision(predictions):
    precision = precision_score(predictions["Target"], predictions["Predictions"])
    print("Precision:", precision)

In [301]:
def display_value_counts(predictions):
    value_counts = predictions["Predictions"].value_counts()
    print("Value Counts of Predictions:")
    print(value_counts)

In [302]:
def display_actual_value_counts(data):
    actual_value_counts = data["Target"].value_counts()
    print("Actual Value Counts:")
    print(actual_value_counts)

In [303]:
sp500 = fetch_stock_data("^GSPC", start_date="1990-01-02")
sp500 = create_target_column(sp500, horizon=1)

# Train the S&P 500 model first
predictors = ["Close", "Volume", "Open", "High", "Low"]
model = train_random_forest_model(sp500, predictors, "Target")

# Process data/train model for additional tickers
# additional_tickers = ["AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "META", "GOOG", "TSLA", "UNH"]
additional_tickers = ["AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "GOOG", "UNH"]
additional_predictions_dfs = []

for ticker in additional_tickers:
    additional_data = fetch_stock_data(ticker, start_date="1990-01-02")
    ticker_predictions_df = generate_additional_ticker_features(sp500, additional_data, horizon=1, ticker=ticker)
    additional_predictions_dfs.append(ticker_predictions_df)

# Merge all additional ticker predictions into the S&P 500 dataframe
for i, ticker in enumerate(additional_tickers):
    sp500 = pd.merge(sp500, additional_predictions_dfs[i], left_index=True, right_index=True)

combined_predictors = predictors + [f"Prediction_{ticker}" for ticker in additional_tickers]

sp500 = sp500.ffill().dropna()

final_model = train_random_forest_model(sp500, combined_predictors, "Target")

backtest_results = backtest(sp500, final_model, combined_predictors)
calculate_precision(backtest_results)
display_value_counts(backtest_results)

Precision: 0.7648401826484018
Value Counts of Predictions:
Predictions
0.0    1487
1.0     876
Name: count, dtype: int64


In [304]:
backtest_results

,Target,Predictions
Date,,
2014-07-25 00:00:00-04:00,1,0.0
2014-07-28 00:00:00-04:00,0,0.0
2014-07-29 00:00:00-04:00,1,1.0
2014-07-30 00:00:00-04:00,0,0.0
2014-07-31 00:00:00-04:00,0,0.0
...,...,...
2023-12-06 00:00:00-05:00,1,1.0
2023-12-07 00:00:00-05:00,1,1.0
2023-12-08 00:00:00-05:00,1,0.0


In [305]:
sp500

,Open,High,Low,Close,Volume,Tomorrow,Target,Prediction_AAPL,Prediction_MSFT,Prediction_AMZN,Prediction_NVDA,Prediction_GOOGL,Prediction_GOOG,Prediction_UNH
Date,,,,,,,,,,,,,,
2004-08-19 00:00:00-04:00,1095.170044,1095.170044,1086.280029,1091.229980,1249400000,1098.349976,1,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2004-08-20 00:00:00-04:00,1091.229980,1100.260010,1089.569946,1098.349976,1199900000,1095.680054,0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2004-08-23 00:00:00-04:00,1098.349976,1101.400024,1094.729980,1095.680054,1021900000,1096.189941,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2004-08-24 00:00:00-04:00,1095.680054,1100.939941,1092.819946,1096.189941,1092500000,1104.959961,1,0.0,1.0,0.0,1.0,1.0,1.0,1.0
2004-08-25 00:00:00-04:00,1096.189941,1106.290039,1093.239990,1104.959961,1192200000,1105.089966,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 00:00:00-05:00,4586.229980,4590.740234,4546.500000,4549.339844,4245680000,4585.589844,1,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2023-12-07 00:00:00-05:00,4568.839844,4590.919922,4565.220215,4585.589844,3818880000,4604.370117,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2023-12-08 00:00:00-05:00,4576.200195,4609.229980,4574.060059,4604.370117,3707010000,4622.439941,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
